<a href="https://colab.research.google.com/github/RogerCL24/ANN/blob/main/5_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas

dataset = pandas.read_csv('train.csv')

In [2]:
dataset.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
dataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# Clean NA values from Age

dataset.Age.fillna(value=dataset.Age.mean(), inplace=True)
dataset.Fare.fillna(value=dataset.Fare.mean(), inplace=True)
dataset.Embarked.fillna(value=(dataset.Embarked.value_counts().idxmax()),inplace=True)

dataset.Survived.fillna(value=1, inplace=True)

In [6]:
# family_size (join the Parch & SibSp columns)

fsiz = pandas.DataFrame(dataset.apply(lambda x: x.SibSp + x.Parch, axis=1), columns=["Family_Size"])
data = dataset.join(fsiz)

In [7]:
# drop useless columns
data.drop('Name', axis=1, inplace=True)
data.drop('Cabin', axis=1, inplace=True)
data.drop('Ticket', axis=1, inplace=True)

data.drop('Parch', axis=1, inplace=True)
data.drop('SibSp', axis=1, inplace=True)

data.drop('PassengerId', axis=1, inplace=True)

In [8]:
data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family_Size
0,0,3,male,22.0,7.2500,S,1
1,1,1,female,38.0,71.2833,C,1
2,1,3,female,26.0,7.9250,S,0
3,1,1,female,35.0,53.1000,S,1
4,0,3,male,35.0,8.0500,S,0


In [9]:
data.dtypes

Survived         int64
Pclass           int64
Sex             object
Age            float64
Fare           float64
Embarked        object
Family_Size      int64
dtype: object

In [10]:
# convert string columns to numeric

for col in data.select_dtypes(exclude=["number"]).columns:
    print("Converting column "+col+"...")
    data[col] = data[col].astype('category')
    print(data[col].cat.categories)
    data[col] = data[col].cat.codes

Converting column Sex...
Index(['female', 'male'], dtype='object')
Converting column Embarked...
Index(['C', 'Q', 'S'], dtype='object')


In [11]:
data.dtypes

Survived         int64
Pclass           int64
Sex               int8
Age            float64
Fare           float64
Embarked          int8
Family_Size      int64
dtype: object

In [12]:
# Y = f(X)

Y = data["Survived"]
X = data.drop("Survived", axis=1)
print(X.shape, Y.shape)

(891, 6) (891,)


In [13]:
# SCIKIT LEARN library

import sklearn

In [14]:
# training and testing data spliting

from sklearn.model_selection import train_test_split

In [15]:
TRAIN_SIZE = 0.8
TESTING_SIZE = 0.2
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=TESTING_SIZE, random_state=42)

In [17]:
print(x_train.shape, y_train.shape,'\n', x_test.shape, y_test.shape)


(712, 6) (712,) 
 (179, 6) (179,)


In [18]:
# DECISION TREE
from sklearn.tree import DecisionTreeClassifier

In [19]:
model_1 = DecisionTreeClassifier(random_state=0)

In [21]:
model_1.fit(x_train, y_train)

DecisionTreeClassifier(random_state=0)

In [22]:
print(model_1.score(x_train, y_train))

0.9803370786516854


In [23]:
# RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier

In [24]:
model_2 = RandomForestClassifier(n_estimators=15) # should be an odd number
model_2.fit(x_train, y_train)

RandomForestClassifier(n_estimators=15)

In [25]:
print(model_2.score(x_train, y_train))

0.973314606741573


In [26]:
# EVALUATION
# testing dataset

print("Decision Tree: ")
print(model_1.score(x_test, y_test))

Decision Tree: 
0.7653631284916201


In [27]:
print("Random Forest: ")
print(model_2.score(x_test, y_test))

Random Forest: 
0.7988826815642458


In [28]:
# Confusion matrix

from sklearn.metrics import confusion_matrix

In [30]:
# Predict data

y_pred_m1 = model_1.predict(x_test)
y_pred_m2 = model_2.predict(x_test)

In [31]:
# Decision Tree

confusion_matrix(y_test, y_pred_m1)

array([[84, 21],
       [21, 53]])

In [32]:
# Random Forest

confusion_matrix(y_test, y_pred_m2)

array([[87, 18],
       [18, 56]])